In [2]:
from sklearn.datasets import load_diabetes

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import random

In [4]:
X,y = load_diabetes(return_X_y=True)

In [6]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [10]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [12]:
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [14]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568968

# My mini batch gradient regressor class from scratch

In [19]:
class MyMBGDRegressor:
    def __init__(self,batch_size,learning_rate=0.01,epoch=50):
        self.batch_size = batch_size
        self.lr = learning_rate
        self.epoch = epoch
        self.coef_ = None
        self.intercept_ = None

    def fit(self,X_train,y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        for i in range(self.epoch):
            for j in range(int(X_train.shape[0]/self.batch_size)):
                idx = random.sample(range(X_train.shape[0]),self.batch_size)

                y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx]- y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)

        print(self.intercept_,"\n",self.coef_)

    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [21]:
mbr = MyMBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epoch=100)

In [23]:
mbr.fit(X_train,y_train)

155.37181320868766 
 [  27.81898806 -142.66752061  454.38380407  303.65230869  -22.94376524
  -89.12865412 -197.40579852  120.61257998  409.57180844  117.31961486]


In [25]:
y_pred = mbr.predict(X_test)

In [27]:
r2_score(y_test,y_pred)

0.4522588618759624

# Sklearn Mini batch (using partial_fit)

In [29]:
from sklearn.linear_model import SGDRegressor

In [31]:
sgd = SGDRegressor(learning_rate='constant',eta0=0.1)

In [35]:
batch_size = 35

for i in range(100):
    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])

In [37]:
sgd.coef_

array([  33.82492573,  -58.57883454,  348.55398248,  234.10811229,
         20.62144559,  -10.54008483, -181.40457479,  135.24257755,
        297.8931214 ,  126.45836893])

In [39]:
sgd.intercept_

array([146.00432151])

In [41]:
y_pred_sgd = sgd.predict(X_test)

In [43]:
r2_score(y_test,y_pred_sgd)

0.42709504764022743